# co-visitation matrix_type_weighted
typeの重み付ける

In [1]:
import os
import sys
import gc
import subprocess
from dotenv import load_dotenv
load_dotenv
sys.path.append(os.getenv('UTILS_PATH'))
from tqdm import tqdm
import multiprocessing
import random
from collections import defaultdict

import pandas as pd
import cudf
import matplotlib.pyplot as plt

In [2]:
SEED = 42
random.seed(SEED)

In [3]:
INPUT_DIR = os.getenv('INPUT_DIR')
OUTPUT_DIR = os.getenv('OUTPUT_DIR')
PREP_DIR = os.getenv("PREP_DIR")

In [4]:
CHUNK_N = 400
type_weight = {"clicks":1, "carts":6, "orders":3}

In [5]:
# データ読み込み
train_sessions = pd.read_pickle(PREP_DIR + "train_sessions.pkl")
test_sessions = pd.read_pickle(PREP_DIR + "test_sessions.pkl")

train_sessions["type"] = train_sessions["type"].map(type_weight)
test_sessions["type"] = test_sessions["type"].map(type_weight)

train_sessions["ts"] = (train_sessions["ts"]/1000).astype("int32")
test_sessions["ts"] = (test_sessions["ts"]/1000).astype("int32")

sessions = pd.concat([train_sessions, test_sessions])

# chunkのグループ分け
sessions["chunk"] = sessions["session"] % CHUNK_N

del train_sessions, test_sessions
gc.collect()

18

In [6]:
dfs_outer = []
dfs_inner = []

pair_df = pd.DataFrame(columns=["aid_x", "aid_y", "cnt"])
for i, chunk_df in tqdm(sessions.groupby("chunk"), total=CHUNK_N):
    df = cudf.from_pandas(chunk_df)
    df = df.merge(df, on="session")
    df = df.loc[((df.ts_x - df.ts_y).abs() < 24 * 60 * 60) & (df.aid_x != df.aid_y)]
    df = df.rename(columns={"type_y": "wt"})
    df = df.groupby(["aid_x", "aid_y"])["wt"].sum().reset_index()
    df = df.sort_values(["aid_x", "wt"], ascending=(True, False))
    df = df[df.groupby("aid_x").cumcount()<30]
    dfs_inner.append(df.to_pandas())

    if i % 100 == 99:
        if len(dfs_inner) > 0:
            pair_df_inner = pd.concat(dfs_inner)
            pair_df_inner = pair_df_inner.groupby(["aid_x", "aid_y"])["wt"].sum().reset_index()
            dfs_outer.append(pair_df_inner)
            dfs_inner = []
            del pair_df_inner
            gc.collect()

if len(dfs_inner) > 0:
    pair_df_inner = pd.concat(dfs_inner)
    pair_df_inner = pair_df_inner.groupby(["aid_x", "aid_y"])["wt"].sum().reset_index()
    dfs_outer.append(pair_df_inner)
    del pair_df_inner
    gc.collect()

del dfs_inner
gc.collect()

pair_df = pd.concat(dfs_outer)
pair_df = pair_df.groupby(["aid_x", "aid_y"])["wt"].sum().reset_index()
pair_df = pair_df.sort_values(["aid_x", "wt"], ascending=(True, False))
pair_df = pair_df.groupby("aid_x").head(20)
pair_df.to_pickle(PREP_DIR + f"co_visitation_matrix_type_weighted.pkl")

100%|██████████| 400/400 [11:49<00:00,  1.77s/it]  
